In [1]:
# Celda 0 — suprimir warnings + tqdm por STDOUT (una sola barra)
import warnings, sys
warnings.filterwarnings("ignore")

try:
    from tqdm import tqdm as _tqdm
    from functools import partial
    tqdm = partial(_tqdm, file=sys.stdout, dynamic_ncols=True, mininterval=0.2, leave=True)
except Exception:
    def tqdm(x, **k):  # fallback si no hay tqdm
        return x

In [2]:
# Celda 1 — Imports, paths y parámetros
# --- Imports
import os, json, joblib
from datetime import timedelta
import numpy as np
import pandas as pd

# --- Paths (ajustá si tu notebook está en otra carpeta)
MODEL_PATH = "../models/03D/best_model.pkl"
DATA_PATH  = "../data/curated/ecobici_model_ready.parquet"
OUT_DIR    = "../predictions"

# --- Parámetros de corrida
ASOF       = "2025-10-24 10:00:00"   # fecha-hora de referencia
HORIZONS   = [1, 3, 6, 12]           # en pasos
FREQ_MIN   = 60                      # minutos por paso (60 = 1h)

# --- Columnas clave
Y_COL  = "num_bikes_available"
ID_COL = "station_id"
TS_COL = "ts_local"

# --- Checks y setup
assert os.path.exists(MODEL_PATH), f"Modelo no encontrado: {MODEL_PATH}"
assert os.path.exists(DATA_PATH),  f"Datos no encontrados: {DATA_PATH}"
os.makedirs(OUT_DIR, exist_ok=True)

print("Listo. Parámetros:")
print("ASOF:", ASOF, "| HORIZONS:", HORIZONS, "| FREQ_MIN:", FREQ_MIN)

Listo. Parámetros:
ASOF: 2025-10-24 10:00:00 | HORIZONS: [1, 3, 6, 12] | FREQ_MIN: 60


In [3]:
# Celda 2 — Utilidades (expandir JSON, one-hot, features temporales)
def _looks_like_json_dict(s: str) -> bool:
    s = str(s).strip()
    return s.startswith("{") and s.endswith("}")

def expand_json_like_columns(df: pd.DataFrame, exclude: list[str]) -> pd.DataFrame:
    """Detecta columnas object con dicts (JSON-like) y las expande a numéricas."""
    df2 = df.copy()
    obj_cols = [c for c in df2.columns if c not in exclude and df2[c].dtype == "object"]
    for col in obj_cols:
        sample = df2[col].dropna().astype(str).head(50)
        if len(sample) == 0:
            continue
        if sample.map(_looks_like_json_dict).mean() >= 0.6:
            def _parse(x):
                try:
                    d = json.loads(x) if isinstance(x, str) else x
                    return d if isinstance(d, dict) else {}
                except Exception:
                    return {}
            exp = df2[col].apply(_parse).apply(pd.Series)
            if exp is not None and exp.shape[1] > 0:
                exp = exp.add_prefix(f"{col}_")
                for c in exp.columns:
                    exp[c] = pd.to_numeric(exp[c], errors="coerce").fillna(0.0)
                df2 = pd.concat([df2.drop(columns=[col]), exp], axis=1)
    return df2

def onehot_low_card(dfX: pd.DataFrame, max_card: int = 20) -> pd.DataFrame:
    low = [c for c in dfX.columns if dfX[c].dtype == "object" and dfX[c].nunique(dropna=True) <= max_card]
    if low:
        dfX = pd.get_dummies(dfX, columns=low, drop_first=True)
    return dfX

def make_numeric_features(dfX: pd.DataFrame) -> pd.DataFrame:
    dfX = expand_json_like_columns(dfX, exclude=[])
    dfX = onehot_low_card(dfX)
    return dfX.select_dtypes(include=["number"]).fillna(0.0)

def time_features_from_ts(ts: pd.Timestamp) -> dict:
    hour  = ts.hour
    dow   = ts.dayofweek
    month = ts.month
    is_weekend = int(dow in (5, 6))
    hour_sin = np.sin(2*np.pi*hour/24)
    hour_cos = np.cos(2*np.pi*hour/24)
    return {
        "hour": hour, "dow": dow, "month": month, "is_weekend": is_weekend,
        "hour_sin": hour_sin, "hour_cos": hour_cos
    }

In [4]:
# Celda 3 — Cargar modelo y datos, armonizar zona horaria, tomar última observación ≤ ASOF por estación
model = joblib.load(MODEL_PATH)

df = pd.read_parquet(DATA_PATH).copy()
df[TS_COL] = pd.to_datetime(df[TS_COL], errors="coerce")
df = df.dropna(subset=[TS_COL])

# Detectar tz de ts_local (puede ser None o una zona, p.ej. America/Argentina/Buenos_Aires)
tz = getattr(df[TS_COL].dt.tz, 'zone', None) or df[TS_COL].dt.tz

# Construir ASOF Timestamp y armonizar:
ASOF_TS = pd.to_datetime(ASOF)
if tz is not None:
    # ts_local es tz-aware → aseguramos que ASOF también lo sea en la misma tz
    if ASOF_TS.tzinfo is None:
        ASOF_TS = ASOF_TS.tz_localize(tz)
    else:
        ASOF_TS = ASOF_TS.tz_convert(tz)
else:
    # ts_local es naive → aseguramos que ASOF también sea naive
    if ASOF_TS.tzinfo is not None:
        ASOF_TS = ASOF_TS.tz_convert(None)

# Ordenar y tomar última fila ≤ ASOF por estación
df = df.sort_values([ID_COL, TS_COL])
latest = (
    df[df[TS_COL] <= ASOF_TS]
    .groupby(ID_COL, as_index=False)
    .tail(1)
    .reset_index(drop=True)
)

print("Estaciones con dato ≤ ASOF:", latest[ID_COL].nunique())
latest.head(3)

Estaciones con dato ≤ ASOF: 393


,station_id,num_bikes_available,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,last_reported,is_charging_station,status,is_installed,...,is_closed,hour,dow,is_weekend,month,hour_sin,hour_cos,y_lag1,y_lag2,y_ma3
0,2,11,"{""mechanical"": 11, ""ebike"": 0}",0,29,0,1759796035,False,IN_SERVICE,1,...,False,21,0,0,10,-0.707107,0.707107,11.0,11.0,11.000000
1,3,4,"{""mechanical"": 4, ""ebike"": 0}",2,22,0,1759796033,False,IN_SERVICE,1,...,False,21,0,0,10,-0.707107,0.707107,4.0,2.0,2.666667
2,4,7,"{""mechanical"": 7, ""ebike"": 0}",6,7,0,1759796049,False,IN_SERVICE,1,...,False,21,0,0,10,-0.707107,0.707107,7.0,7.0,7.000000


In [5]:
# Celda 3b — Obtener esquema de columnas EXACTO usado en el entrenamiento
TRAIN_SPLIT = "../data/splits/train.parquet"  # ajusta si tu notebook está en otra carpeta
df_train_for_schema = pd.read_parquet(TRAIN_SPLIT)

# Reutilizamos la MISMA función make_X_y del notebook del entrenamiento
# (si no la tenés en este notebook, copia la definición usada en train)
def _looks_like_json_dict(s: str) -> bool:
    s = str(s).strip()
    return s.startswith("{") and s.endswith("}")

def expand_json_like_columns(df: pd.DataFrame, exclude: list[str]) -> pd.DataFrame:
    df2 = df.copy()
    obj_cols = [c for c in df2.columns if c not in exclude and df2[c].dtype == "object"]
    for col in obj_cols:
        sample = df2[col].dropna().astype(str).head(50)
        if len(sample) == 0:
            continue
        if sample.map(_looks_like_json_dict).mean() >= 0.6:
            import json
            def _parse(x):
                try:
                    d = json.loads(x) if isinstance(x, str) else x
                    return d if isinstance(d, dict) else {}
                except Exception:
                    return {}
            exp = df2[col].apply(_parse).apply(pd.Series)
            if exp is not None and exp.shape[1] > 0:
                exp = exp.add_prefix(f"{col}_")
                for c in exp.columns:
                    exp[c] = pd.to_numeric(exp[c], errors="coerce").fillna(0.0)
                df2 = pd.concat([df2.drop(columns=[col]), exp], axis=1)
    return df2

def onehot_low_card(dfX: pd.DataFrame, max_card: int = 20) -> pd.DataFrame:
    low = [c for c in dfX.columns if dfX[c].dtype == "object" and dfX[c].nunique(dropna=True) <= max_card]
    if low:
        dfX = pd.get_dummies(dfX, columns=low, drop_first=True)
    return dfX

def make_X_y_train_schema(df: pd.DataFrame, y_col: str, id_col: str, ts_col: str):
    drop_cols = [c for c in [y_col, id_col, ts_col] if c in df.columns]
    X = df.drop(columns=drop_cols, errors="ignore").copy()
    X = expand_json_like_columns(X, exclude=[])
    X = onehot_low_card(X)
    X = X.select_dtypes(include=["number"]).fillna(0.0)
    y = df[y_col].astype(float).copy()
    return X, y

X_schema, _ = make_X_y_train_schema(df_train_for_schema, Y_COL, ID_COL, TS_COL)
TRAIN_COLS = list(X_schema.columns)    # ← columna y orden exactos del entrenamiento
len(TRAIN_COLS), TRAIN_COLS[:10]

(22,
 ['num_bikes_disabled',
  'num_docks_available',
  'num_docks_disabled',
  'last_reported',
  'is_installed',
  'is_renting',
  'is_returning',
  '_file_last_updated',
  'lat',
  'lon'])

In [6]:
# Celda 4 — Preparar “constantes por estación” (numéricas) para reusar en cada horizonte
latest_num = latest.copy()
latest_num = latest_num.drop(columns=[c for c in [Y_COL, ID_COL, TS_COL] if c in latest_num.columns], errors="ignore")
latest_num = make_numeric_features(latest_num)

# Indexar por station_id para lookup directo
latest_num.index = latest[ID_COL].values

print("Columnas numéricas (constantes por estación):", len(latest_num.columns))
list(latest_num.columns)[:10]

Columnas numéricas (constantes por estación): 22


['num_bikes_disabled',
 'num_docks_available',
 'num_docks_disabled',
 'last_reported',
 'is_installed',
 'is_renting',
 'is_returning',
 '_file_last_updated',
 'lat',
 'lon']

In [7]:
# Celda 5 — una sola barra de progreso (exterior)
preds = []

total_est = len(latest)
for i in tqdm(range(total_est), desc="🔄 Estaciones"):
    row = latest.iloc[i]
    sid = row[ID_COL]

    y_lag1 = float(row.get("y_lag1", float("nan")))
    y_lag2 = float(row.get("y_lag2", float("nan")))
    y_ma3  = float(row.get("y_ma3", float("nan")))
    if np.isnan(y_lag1) or np.isnan(y_lag2) or np.isnan(y_ma3):
        continue

    cur_ts = ASOF_TS
    const_feats = latest_num.loc[sid].astype(float).to_dict() if sid in latest_num.index else {}

    # podés mostrar info breve en la barra
    # (ej: último horizonte procesado, estación, etc.)
    # tqdm_instance = tqdm(...) devolvería un objeto, pero como usamos "partial" arriba, hacemos:
    # no es necesario; si querés post-fijo dinámico:
    # tqdm.set_postfix_str(f"sid={sid}")

    for h in HORIZONS:
        ts_pred = cur_ts + timedelta(minutes=FREQ_MIN * h)

        feats = {
            **time_features_from_ts(ts_pred),
            "y_lag1": y_lag1, "y_lag2": y_lag2, "y_ma3": y_ma3,
            **const_feats
        }

        X_raw = pd.DataFrame([feats]).select_dtypes(include=["number"]).fillna(0.0)
        X = X_raw.reindex(columns=TRAIN_COLS, fill_value=0.0)

        yhat = float(model.predict(X)[0])
        yhat_lo = yhat_hi = None
        if hasattr(model, "estimators_") and isinstance(model.estimators_, list) and len(model.estimators_) > 1:
            trees = np.array([t.predict(X)[0] for t in model.estimators_], dtype=float)
            std = float(np.std(trees))
            yhat_lo = yhat - 1.96*std
            yhat_hi = yhat + 1.96*std

        preds.append({
            ID_COL: sid,
            "timestamp_pred": ts_pred,
            "h": int(h),
            "yhat": yhat,
            "yhat_lo": yhat_lo,
            "yhat_hi": yhat_hi
        })

        # actualización recursiva de lags
        y_lag2 = y_lag1
        y_lag1 = yhat
        y_ma3  = float(np.mean([yhat, y_lag2, y_ma3]))

🔄 Estaciones: 100%|██████████████████████████| 393/393 [05:11<00:00,  1.26it/s]


In [8]:
# Celda 6 — Resultado y guardado
if not preds:
    raise RuntimeError("No se generaron predicciones (posible falta de lags en todas las estaciones).")

df_pred = pd.DataFrame(preds).sort_values([ID_COL, "timestamp_pred", "h"]).reset_index(drop=True)
out_path = os.path.join(OUT_DIR, "predictions.parquet")
df_pred.to_parquet(out_path, index=False)

print(f"[OK] Predicciones guardadas en: {out_path}")
df_pred.head(10)

[OK] Predicciones guardadas en: ../predictions/predictions.parquet


,station_id,timestamp_pred,h,yhat,yhat_lo,yhat_hi
0,2,2025-10-24 11:00:00-03:00,1,11.0,11.0,11.0
1,2,2025-10-24 13:00:00-03:00,3,11.0,11.0,11.0
2,2,2025-10-24 16:00:00-03:00,6,11.0,11.0,11.0
3,2,2025-10-24 22:00:00-03:00,12,11.0,11.0,11.0
4,3,2025-10-24 11:00:00-03:00,1,4.0,4.0,4.0
5,3,2025-10-24 13:00:00-03:00,3,4.0,4.0,4.0
6,3,2025-10-24 16:00:00-03:00,6,4.0,4.0,4.0
7,3,2025-10-24 22:00:00-03:00,12,4.0,4.0,4.0
8,4,2025-10-24 11:00:00-03:00,1,7.0,7.0,7.0
9,4,2025-10-24 13:00:00-03:00,3,7.0,7.0,7.0


📘 Descripción de los datos de salida (predictions.parquet)

El archivo generado en el Paso 4 contiene las predicciones multi-horizonte de disponibilidad de bicicletas por estación, obtenidas a partir del último registro disponible antes del instante de referencia ASOF.

Cada fila representa una predicción puntual (y opcionalmente un intervalo) para una estación específica en un horizonte temporal determinado.

| Columna	|Tipo	|Descripción |
|-----------|-------|------------|
|station_id	int / str	|Identificador único de la estación Ecobici. |Se corresponde con el mismo station_id presente en los datos históricos. |
|timestamp_pred	datetime	|Marca de tiempo (en formato local) que indica el momento futuro al que se refiere la predicción. |Se calcula como ASOF + h × FREQ_MIN. |
|h	|int	|Horizonte temporal expresado en pasos. Cada paso equivale a FREQ_MIN minutos (por defecto, 60 min). Ejemplo: h=3 → predicción a 3 horas.|
|yhat	|float	|Predicción puntual del modelo: número estimado de bicicletas disponibles en la estación en ese instante futuro. |
|yhat_lo	|float (opcional)	|Límite inferior del intervalo de confianza aproximado (solo disponible si el modelo es un ensamble, ej. RandomForest). |
|yhat_hi	|float (opcional)	|Límite superior del intervalo de confianza aproximado. |
|(meta)		| |El dataset incluye una fila por combinación (station_id, h) para cada timestamp_pred. |


⸻

🔍 Interpretación y uso en el dashboard
- Eje temporal: timestamp_pred
Permite graficar series de predicciones futuras (ej. próximas 12 horas) para cada estación.
- Mapa de calor / Semáforo:
    - Variable principal: yhat (bicis disponibles).
    - Se pueden definir rangos de color:
        - 🟥 0–2 bicis → “crítica”
        - 🟨 3–5 bicis → “baja disponibilidad”
        - 🟩 > 5 bicis → “normal”
    - Estos umbrales pueden ajustarse dinámicamente en el dashboard.
- Horizontes:
    - Si el usuario selecciona un horizonte (p. ej. 3 horas), se filtran las filas con h == 3.
- Intervalos de confianza:
    - Si el dashboard incluye barras de error o transparencia, yhat_lo y yhat_hi permiten mostrar la incertidumbre asociada.
- Integración geográfica:
    - Para construir un mapa interactivo (Leaflet / Plotly / Folium), se puede unir esta tabla con station_information.parquet usando station_id para obtener latitud, longitud y nombre de estación.

⸻

💡 Ejemplo de unión para visualización:
```
st_info = pd.read_parquet("../data/curated/station_information.parquet")
df_map = df_pred.merge(st_info, on="station_id", how="left")
```

Así tendrás un dataframe con:
station_id, name, lat, lon, timestamp_pred, h, yhat →
listo para alimentar un heatmap o dashboard de disponibilidad futura.